In [ ]:
# Environment
import numpy as np
import pandas as pd
from MesoPy import Meso
import matplotlib.pyplot as plt
from datetime import datetime, timedelta, time
import pytz
# Local modules for handling data and running moisture models
import data_funcs as datf
from data_funcs import format_precip, fixnan
import moisture_models as mod

meso_token="4192c18707b848299783d59a9317c6e1"
m=Meso(meso_token)

## Retrieve RAWS Function

In [ ]:
raws_vars='air_temp,relative_humidity,precip_accum,fuel_moisture'
time_start = "201806010800"  # June 1 2018 08:00 in format yyyymmddHHMM
time_end   = "201907200900"  # June 20 2018 09:00 in format yyyymmddHHMM 

In [ ]:
def format_raws(stn, fix_names = True):
    raws_dat = stn['OBSERVATIONS']
    
    # Convert to Numpy arrays, check data type for floats
    for key in [*stn['OBSERVATIONS'].keys()]:
        if type(stn['OBSERVATIONS'][key][0]) is float:
            raws_dat[key] = np.array(stn['OBSERVATIONS'][key], dtype = 'float64')
        else:
            raws_dat[key] = np.array(stn['OBSERVATIONS'][key])
    
    # Transform Data
    raws_dat['air_temp_set_1'] = raws_dat['air_temp_set_1'] + 273.15 ## convert C to K
    if 'precip_accum_set_1' in raws_dat.keys():
        raws_dat['precip_accum_set_1'] = format_precip(raws_dat['precip_accum_set_1']) ## format precip data, accumulated to hourly
    
    
    # Calculate Equilibrium Temps
    raws_dat['Ed'] = 0.924*raws_dat['relative_humidity_set_1']**0.679 + 0.000499*np.exp(0.1*raws_dat['relative_humidity_set_1']) + 0.18*(21.1 + 273.15 - raws_dat['air_temp_set_1'])*(1 - np.exp(-0.115*raws_dat['relative_humidity_set_1']))
    raws_dat['Ew'] = 0.618*raws_dat['relative_humidity_set_1']**0.753 + 0.000454*np.exp(0.1*raws_dat['relative_humidity_set_1']) + 0.18*(21.1 + 273.15 - raws_dat['air_temp_set_1'])*(1 - np.exp(-0.115*raws_dat['relative_humidity_set_1']))
    
    # Fix nan values
    for key in [*raws_dat.keys()]:
        if type(raws_dat[key][0]) is float:
            raws_dat[key] = fixnan(raws_dat[key], 2)
    
    return raws_dat

In [ ]:
def retrieve_raws(stid, raws_vars, time1, time2):
    meso_ts = m.timeseries(time1, time2, 
                       stid=stid, vars=raws_vars)
    station = meso_ts['STATION'][0]
    
    raws_dat = format_raws(station)
    
    return station, raws_dat

In [ ]:
raws_vars='air_temp,relative_humidity,precip_accum,fuel_moisture'
time_start = "201806010800"  # June 1 2018 08:00 in format yyyymmddHHMM
time_end   = "201907200900"  # June 20 2018 09:00 in format yyyymmddHHMM 

In [ ]:
station, raws_dat = retrieve_raws("CPTC2", raws_vars, time_start, time_end)

In [ ]:
def plot_dat(stn, dat, val):
    plt.figure(figsize=(16,4))
    plt.plot(dat[val],linestyle='-',c='k')
    plt.title(stn['STID']+' '+ val)
    plt.xlabel('Time (hours)') 
    plt.ylabel('val')

In [ ]:
%matplotlib inline
plot_dat(station, raws_dat, 'fuel_moisture_set_1')

## Retrieve RTMA Function

## Fit RNN Model

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt
import tensorflow as tf
import keras.backend as K
from keras.utils.vis_utils import plot_model
from scipy.interpolate import LinearNDInterpolator, interpn
from scipy.optimize import root

## Local Modules
from moisture_models import create_RNN, create_RNN_2, staircase, seq2batches

In [ ]:
def RELU(x):
    if x>0. :
        return x
    else:
        return 0.

# network computing z = a*x1 + b*x2 with offset c
def linrelu(x,a,b,c):
    y = np.dot(np.array([[a, b], [-a, -b] ]), x) + np.array([c, -c])
    y[0]=RELU(y[0])
    y[1]=RELU(y[1])
    return(np.dot([1,-1],y))-c

In [ ]:
# Set seed for reproducibility
tf.random.set_seed(123)

In [ ]:
# Define Data for Model
Ew = raws_dat['Ew']
Ed = raws_dat['Ew']
rain = raws_dat['precip_accum_set_1']
fm = raws_dat['fuel_moisture_set_1']
temp = raws_dat['air_temp_set_1']
h2 = 300

# Average Equilibrium
E = (Ed + Ew)/2
first_rain=np.nonzero(rain>0)[0][0]
print(first_rain)
hours=first_rain
E=E[:hours]
data=fm[:hours]

# transform as 2D, (timesteps, features) and (timesteps, outputs)
Et = np.reshape(E,[E.shape[0],1])
datat = np.reshape(data,[data.shape[0],1])

# Scale Data if required
scale=False
if scale:
    scalerx = MinMaxScaler()
    scalerx.fit(Et)
    Et = scalerx.transform(Et)
    scalery = MinMaxScaler()
    scalery.fit(datat)
    datat = scalery.transform(datat)

In [ ]:
# split data
return_sequences=False
x_train, y_train = staircase(Et,datat,timesteps=5,trainsteps=h2,
                             return_sequences=return_sequences)
print('x_train shape=',x_train.shape)
samples, timesteps, features = x_train.shape
print('y_train shape=',y_train.shape)
# the simplest model possible
activation=['linear','linear']
hidden_units=3
dense_units=1
dense_layers=1
features=1
hours=Et.shape[0]
h0 = tf.convert_to_tensor(datat[:samples],dtype=tf.float32)
shift = 0.

# statefull model version for traning
model_fit=create_RNN_2(hidden_units=hidden_units, 
                        dense_units=dense_units, 
                        batch_shape=(samples,timesteps,features),
                        stateful=True,
                        return_sequences=return_sequences,
                        # initial_state=h0,
                        activation=activation,
                        dense_layers=dense_layers)

In [ ]:
model_predict=create_RNN_2(hidden_units=hidden_units, dense_units=dense_units,  
                        input_shape=(hours,features),stateful = False,
                        return_sequences=True,
                        activation=activation,dense_layers=dense_layers)
# model_predict=create_RNN_sequences(hidden_units=1, dense_units=1, input_shape=(hours,1), 
#                        activation=['linear', 'linear'])
print('model_predict input shape',Et.shape,'output shape',model_predict(Et).shape)
print(model_predict.summary())

In [ ]:
# fitting
DeltaE = 0
w_exact=  [np.array([[1.-np.exp(-0.1)]]), np.array([[np.exp(-0.1)]]), np.array([0.]),np.array([[1.0]]),np.array([-1.*DeltaE])]
w_initial=[np.array([[1.-np.exp(-0.1)]]), np.array([[np.exp(-0.1)]]), np.array([0.]),np.array([[1.0]]),np.array([-1.0])]
w=model_fit.get_weights()
for i in range(len(w)):
  print('weight',i,'shape',w[i].shape,'ndim',w[i].ndim,'given',w_initial[i].shape)
  for j in range(w[i].shape[0]):
    if w[i].ndim==2:
      for k in range(w[i].shape[1]):
        w[i][j][k]=w_initial[i][0][0]/w[i].shape[0]
    else:
      w[i][j]=w_initial[i][0]
model_fit.set_weights(w)
model_fit.fit(x_train, y_train, epochs=5000, verbose=0,batch_size=samples)
w_fitted=model_fit.get_weights()
for i in range(len(w)):
  print('weight',i,' exact:',w_exact[i],':  initial:',w_initial[i],' fitted:',w_fitted[i])

In [ ]:
model_predict.set_weights(w_fitted)
x_input=np.reshape(Et,(1, hours, 1))
y_output = model_predict.predict(x_input)
print('x_input.shape=',x_input.shape,'y_output.shape=',y_output.shape)
print(shift)
m = np.reshape(y_output,hours)
print('weights=',w)
if scale:
    print('scaling')
    m = scalery.inverse_transform(m)
m = np.reshape(m,hours)
hour=np.array(range(hours))
title="First RNN forecast"
plt.figure(figsize=(16,4))
plt.plot(hour,E,linestyle='--',c='r',label='E=Equilibrium data')
# print(len(hour),len(m_f))
plt.scatter(hour,data,c='b',label='data=10-h fuel data')
if m is not None:
    plt.plot(hour[:h2],m[:h2],linestyle='-',c='k',label='m=filtered')
    plt.plot(hour[h2:hours],m[h2:hours],linestyle='-',c='r',label='m=forecast')
plt.title(title) 
plt.legend()

In [ ]:
hours